In [1]:
import pandas as pd
from path import Path
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pymongo
import pandas as pd
import s3fs
import datetime

In [2]:
app_data = pd.read_csv('s3://pgchirclassadmin/Google_App_Store_Data_Clean.csv')
app_data.head()

,Unnamed: 0,App Name,Category,Rating,Rating Count,Maximum Installs,Price,Size (Mb),Released,Ad Supported,In App Purchases
0,1,Ampere Battery Info,Tools,4.4,64,7662,0.0,2.9,5/21/2020,1,0
1,9,Neon 3d Iron Tech Keyboard Theme,Personalization,4.7,820,62433,0.0,3.5,9/22/2019,1,0
2,10,Dodge The Cars!,Racing,4.9,55,329,0.0,51,7/30/2020,0,0
3,12,桃園機場捷運時刻表 - 捷運轉乘路線快速查詢(支援台北捷運),Travel & Local,3.9,118,37763,0.0,7.6,4/3/2018,1,0
4,14,Caliway Conductor,Travel & Local,3.7,1572,42856,0.0,29,9/5/2018,0,0


In [3]:
app_data = app_data.set_index('App Name')
app_data.head()

,Unnamed: 0,Category,Rating,Rating Count,Maximum Installs,Price,Size (Mb),Released,Ad Supported,In App Purchases
App Name,,,,,,,,,,
Ampere Battery Info,1,Tools,4.4,64,7662,0.0,2.9,5/21/2020,1,0
Neon 3d Iron Tech Keyboard Theme,9,Personalization,4.7,820,62433,0.0,3.5,9/22/2019,1,0
Dodge The Cars!,10,Racing,4.9,55,329,0.0,51,7/30/2020,0,0
桃園機場捷運時刻表 - 捷運轉乘路線快速查詢(支援台北捷運),12,Travel & Local,3.9,118,37763,0.0,7.6,4/3/2018,1,0
Caliway Conductor,14,Travel & Local,3.7,1572,42856,0.0,29,9/5/2018,0,0


In [4]:
app_data['Released'] = pd.to_datetime(app_data['Released'])
app_data.head()

,Unnamed: 0,Category,Rating,Rating Count,Maximum Installs,Price,Size (Mb),Released,Ad Supported,In App Purchases
App Name,,,,,,,,,,
Ampere Battery Info,1,Tools,4.4,64,7662,0.0,2.9,2020-05-21,1,0
Neon 3d Iron Tech Keyboard Theme,9,Personalization,4.7,820,62433,0.0,3.5,2019-09-22,1,0
Dodge The Cars!,10,Racing,4.9,55,329,0.0,51,2020-07-30,0,0
桃園機場捷運時刻表 - 捷運轉乘路線快速查詢(支援台北捷運),12,Travel & Local,3.9,118,37763,0.0,7.6,2018-04-03,1,0
Caliway Conductor,14,Travel & Local,3.7,1572,42856,0.0,29,2018-09-05,0,0


In [5]:
app_data['Month_Num'] = app_data['Released'].dt.month
app_data.head()

,Unnamed: 0,Category,Rating,Rating Count,Maximum Installs,Price,Size (Mb),Released,Ad Supported,In App Purchases,Month_Num
App Name,,,,,,,,,,,
Ampere Battery Info,1,Tools,4.4,64,7662,0.0,2.9,2020-05-21,1,0,5
Neon 3d Iron Tech Keyboard Theme,9,Personalization,4.7,820,62433,0.0,3.5,2019-09-22,1,0,9
Dodge The Cars!,10,Racing,4.9,55,329,0.0,51,2020-07-30,0,0,7
桃園機場捷運時刻表 - 捷運轉乘路線快速查詢(支援台北捷運),12,Travel & Local,3.9,118,37763,0.0,7.6,2018-04-03,1,0,4
Caliway Conductor,14,Travel & Local,3.7,1572,42856,0.0,29,2018-09-05,0,0,9


In [6]:
app_data = app_data.drop(["Unnamed: 0"],axis=1)
app_data.head()

,Category,Rating,Rating Count,Maximum Installs,Price,Size (Mb),Released,Ad Supported,In App Purchases,Month_Num
App Name,,,,,,,,,,
Ampere Battery Info,Tools,4.4,64,7662,0.0,2.9,2020-05-21,1,0,5
Neon 3d Iron Tech Keyboard Theme,Personalization,4.7,820,62433,0.0,3.5,2019-09-22,1,0,9
Dodge The Cars!,Racing,4.9,55,329,0.0,51,2020-07-30,0,0,7
桃園機場捷運時刻表 - 捷運轉乘路線快速查詢(支援台北捷運),Travel & Local,3.9,118,37763,0.0,7.6,2018-04-03,1,0,4
Caliway Conductor,Travel & Local,3.7,1572,42856,0.0,29,2018-09-05,0,0,9


In [7]:
app_data['Rating'].median()

4.2

In [8]:
app_data['Maximum Installs'].median()

43901.5

In [9]:
app_data['Good_App'] = np.where((app_data['Rating']>=4.2) & (app_data['Rating Count']>=250) & (app_data['Maximum Installs']>=43901.5),1,0)
app_data.head()

,Category,Rating,Rating Count,Maximum Installs,Price,Size (Mb),Released,Ad Supported,In App Purchases,Month_Num,Good_App
App Name,,,,,,,,,,,
Ampere Battery Info,Tools,4.4,64,7662,0.0,2.9,2020-05-21,1,0,5,0
Neon 3d Iron Tech Keyboard Theme,Personalization,4.7,820,62433,0.0,3.5,2019-09-22,1,0,9,1
Dodge The Cars!,Racing,4.9,55,329,0.0,51,2020-07-30,0,0,7,0
桃園機場捷運時刻表 - 捷運轉乘路線快速查詢(支援台北捷運),Travel & Local,3.9,118,37763,0.0,7.6,2018-04-03,1,0,4,0
Caliway Conductor,Travel & Local,3.7,1572,42856,0.0,29,2018-09-05,0,0,9,0


In [10]:
app_data_encoded = pd.get_dummies(app_data, columns=['Category'])
app_data_encoded.head()

,Rating,Rating Count,Maximum Installs,Price,Size (Mb),Released,Ad Supported,In App Purchases,Month_Num,Good_App,...,Category_Simulation,Category_Social,Category_Sports,Category_Strategy,Category_Tools,Category_Travel & Local,Category_Trivia,Category_Video Players & Editors,Category_Weather,Category_Word
App Name,,,,,,,,,,,,,,,,,,,,,
Ampere Battery Info,4.4,64,7662,0.0,2.9,2020-05-21,1,0,5,0,...,0,0,0,0,1,0,0,0,0,0
Neon 3d Iron Tech Keyboard Theme,4.7,820,62433,0.0,3.5,2019-09-22,1,0,9,1,...,0,0,0,0,0,0,0,0,0,0
Dodge The Cars!,4.9,55,329,0.0,51,2020-07-30,0,0,7,0,...,0,0,0,0,0,0,0,0,0,0
桃園機場捷運時刻表 - 捷運轉乘路線快速查詢(支援台北捷運),3.9,118,37763,0.0,7.6,2018-04-03,1,0,4,0,...,0,0,0,0,0,1,0,0,0,0
Caliway Conductor,3.7,1572,42856,0.0,29,2018-09-05,0,0,9,0,...,0,0,0,0,0,1,0,0,0,0


In [11]:
app_data_encoded.columns

Index(['Rating', 'Rating Count', 'Maximum Installs', 'Price', 'Size (Mb)',
       'Released', 'Ad Supported', 'In App Purchases', 'Month_Num', 'Good_App',
       'Category_Action', 'Category_Adventure', 'Category_Arcade',
       'Category_Art & Design', 'Category_Auto & Vehicles', 'Category_Beauty',
       'Category_Board', 'Category_Books & Reference', 'Category_Business',
       'Category_Card', 'Category_Casino', 'Category_Casual',
       'Category_Comics', 'Category_Communication', 'Category_Dating',
       'Category_Education', 'Category_Educational', 'Category_Entertainment',
       'Category_Events', 'Category_Finance', 'Category_Food & Drink',
       'Category_Health & Fitness', 'Category_House & Home',
       'Category_Libraries & Demo', 'Category_Lifestyle',
       'Category_Maps & Navigation', 'Category_Medical', 'Category_Music',
       'Category_Music & Audio', 'Category_News & Magazines',
       'Category_Parenting', 'Category_Personalization',
       'Category_Photograph

In [12]:
app_data_encoded = app_data_encoded.drop(['Released','Size (Mb)'],axis=1)
app_data_encoded.head()

,Rating,Rating Count,Maximum Installs,Price,Ad Supported,In App Purchases,Month_Num,Good_App,Category_Action,Category_Adventure,...,Category_Simulation,Category_Social,Category_Sports,Category_Strategy,Category_Tools,Category_Travel & Local,Category_Trivia,Category_Video Players & Editors,Category_Weather,Category_Word
App Name,,,,,,,,,,,,,,,,,,,,,
Ampere Battery Info,4.4,64,7662,0.0,1,0,5,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Neon 3d Iron Tech Keyboard Theme,4.7,820,62433,0.0,1,0,9,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Dodge The Cars!,4.9,55,329,0.0,0,0,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
桃園機場捷運時刻表 - 捷運轉乘路線快速查詢(支援台北捷運),3.9,118,37763,0.0,1,0,4,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Caliway Conductor,3.7,1572,42856,0.0,0,0,9,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [13]:
app_data_encoded.dtypes

Rating                              float64
Rating Count                          int64
Maximum Installs                      int64
Price                               float64
Ad Supported                          int64
In App Purchases                      int64
Month_Num                             int64
Good_App                              int32
Category_Action                       uint8
Category_Adventure                    uint8
Category_Arcade                       uint8
Category_Art & Design                 uint8
Category_Auto & Vehicles              uint8
Category_Beauty                       uint8
Category_Board                        uint8
Category_Books & Reference            uint8
Category_Business                     uint8
Category_Card                         uint8
Category_Casino                       uint8
Category_Casual                       uint8
Category_Comics                       uint8
Category_Communication                uint8
Category_Dating                 

In [14]:
data_scaler = StandardScaler()

In [15]:
app_data_scaled = data_scaler.fit_transform(app_data_encoded)

In [16]:
X = app_data_encoded.copy()
X = X.drop(["Rating","Rating Count","Maximum Installs","Good_App"], axis=1)
X.head()

,Price,Ad Supported,In App Purchases,Month_Num,Category_Action,Category_Adventure,Category_Arcade,Category_Art & Design,Category_Auto & Vehicles,Category_Beauty,...,Category_Simulation,Category_Social,Category_Sports,Category_Strategy,Category_Tools,Category_Travel & Local,Category_Trivia,Category_Video Players & Editors,Category_Weather,Category_Word
App Name,,,,,,,,,,,,,,,,,,,,,
Ampere Battery Info,0.0,1,0,5,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Neon 3d Iron Tech Keyboard Theme,0.0,1,0,9,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dodge The Cars!,0.0,0,0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
桃園機場捷運時刻表 - 捷運轉乘路線快速查詢(支援台北捷運),0.0,1,0,4,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Caliway Conductor,0.0,0,0,9,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [17]:
y = app_data_encoded["Good_App"].values
y[:5]

array([0, 1, 0, 0, 0])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [19]:
# Creating a StandardScaler instance
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

# Scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [21]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

In [22]:
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,103136,612
Actual 1,25375,401


In [24]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [25]:
acc_score

0.7993653685803404

In [26]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,103136,612
Actual 1,25375,401


Accuracy Score : 0.7993653685803404
Classification Report
              precision    recall  f1-score   support

           0       0.80      0.99      0.89    103748
           1       0.40      0.02      0.03     25776

    accuracy                           0.80    129524
   macro avg       0.60      0.50      0.46    129524
weighted avg       0.72      0.80      0.72    129524



In [27]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [28]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [29]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [30]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,103106,642
Actual 1,25326,450


In [31]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [32]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,103106,642
Actual 1,25326,450


Accuracy Score : 0.799512059541089
Classification Report
              precision    recall  f1-score   support

           0       0.80      0.99      0.89    103748
           1       0.41      0.02      0.03     25776

    accuracy                           0.80    129524
   macro avg       0.61      0.51      0.46    129524
weighted avg       0.73      0.80      0.72    129524



In [33]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.07862582, 0.13101387, 0.33180223, 0.18987485, 0.00781567,
       0.00811191, 0.00852357, 0.00145931, 0.00185943, 0.00132539,
       0.00194788, 0.02737523, 0.0102089 , 0.00400379, 0.00919688,
       0.0032802 , 0.00171164, 0.00176301, 0.00294106, 0.00656835,
       0.00189073, 0.00838746, 0.00147894, 0.00296919, 0.00205458,
       0.00915795, 0.00144329, 0.00147648, 0.01017107, 0.00261276,
       0.00188773, 0.00245063, 0.00857957, 0.00935937, 0.00260471,
       0.03693487, 0.00347188, 0.00234481, 0.00686178, 0.00604081,
       0.00409007, 0.00215836, 0.0118312 , 0.00184746, 0.00422091,
       0.00283432, 0.00292851, 0.00219272, 0.00166218, 0.00209577,
       0.00435432, 0.00819658])

In [34]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.3318022342840337, 'In App Purchases'),
 (0.18987484572908628, 'Month_Num'),
 (0.13101387417592478, 'Ad Supported'),
 (0.0786258191088284, 'Price'),
 (0.03693487172406797, 'Category_Personalization'),
 (0.0273752312275328, 'Category_Books & Reference'),
 (0.011831195726805226, 'Category_Simulation'),
 (0.010208903856729989, 'Category_Business'),
 (0.010171070248226017, 'Category_Lifestyle'),
 (0.009359366880773424, 'Category_News & Magazines'),
 (0.009196881129069974, 'Category_Casino'),
 (0.009157953553130644, 'Category_Health & Fitness'),
 (0.008579568385155533, 'Category_Music & Audio'),
 (0.008523569213642767, 'Category_Arcade'),
 (0.008387457574533925, 'Category_Entertainment'),
 (0.00819658254280777, 'Category_Word'),
 (0.008111912743294945, 'Category_Adventure'),
 (0.007815670777104532, 'Category_Action'),
 (0.006861782960365443, 'Category_Puzzle'),
 (0.0065683503863207034, 'Category_Education'),
 (0.00604081214458684, 'Category_Racing'),
 (0.004354315178700579, 'Category_Wea

In [35]:
#Random Undersampling
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 77945, 1: 77945})

In [36]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

C:\ProgramData\Miniconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [37]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[67996, 35752],
       [11836, 13940]], dtype=int64)

In [38]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5981044630968124

In [39]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.66      0.54      0.74      0.60      0.36    103748
          1       0.28      0.54      0.66      0.37      0.60      0.35     25776

avg / total       0.74      0.63      0.56      0.67      0.60      0.36    129524



In [40]:
# Boosting
from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
    learning_rate=learning_rate,
    max_features=5,
    max_depth=3,
    random_state=0)
    classifier.fit(X_train_scaled, y_train.ravel())
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test)))

Learning rate:  0.05
Accuracy score (training): 0.799
Accuracy score (validation): 0.801
Learning rate:  0.1
Accuracy score (training): 0.799
Accuracy score (validation): 0.801
Learning rate:  0.25
Accuracy score (training): 0.799
Accuracy score (validation): 0.801
Learning rate:  0.5
Accuracy score (training): 0.799
Accuracy score (validation): 0.801
Learning rate:  0.75
Accuracy score (training): 0.799
Accuracy score (validation): 0.801
Learning rate:  1
Accuracy score (training): 0.799
Accuracy score (validation): 0.800


In [41]:
classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=0.5, max_features=5, max_depth=3, random_state=0)

classifier.fit(X_train_scaled, y_train)
predictions = classifier.predict(X_test_scaled)

In [42]:
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.8009789691485748


In [43]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
   cm, index=["Actual 0", "Actual 1"],
   columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,103710,38
Actual 1,25740,36
